In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.autograd import Function
import numpy as np


from data import *
from binarizers import *
from models import *

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed(1)

In [14]:
epochs = 10
lr = 1e-3
optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [15]:
def train(epochs):
    net.train()
    
    for epoch in range(1, epochs + 1):
        for idx, (data, target) in enumerate(train_loader):
            if device == 'cuda':
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)

            optimizer.zero_grad()
            output = net(data)
            loss = criterion(output, target)

            if epoch % 40 == 0:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']*0.1
            
            optimizer.zero_grad()
            loss.backward()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.data.copy_(p.org)
            
            optimizer.step()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.org.copy_(p.data.clamp_(-1,1))
            
            with torch.no_grad():
                if idx % 10 == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, idx * len(data), len(train_loader.dataset),
                        100. * idx / len(train_loader), loss.item()))
        test()

def test():
    net.eval()
    test_loss, correct = 0, 0
    
    for data, target in test_loader:
        if device == 'cuda':
            data, target = data.cuda(), target.cuda()
             
        data, target = Variable(data), Variable(target)

        output = net(data)

        test_loss += criterion(output, target).item()

        pred = torch.max(output, dim=1)[1]

        correct += (pred == target).sum()
    
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

train(epochs)

/datasets/home/home-01/49/249/abiyer/binarized-neural-network/models.py:64: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.logsoftmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299698
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.587816
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.437299
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.557898
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.308380
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.422659
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.541240
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.708327
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.189275
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.263072
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.324816
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.640443
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.305461
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.554235
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.542889
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.527236
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.387055
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.189063
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.321044
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.382005
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.335451
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.141466
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.137256
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.155940
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.253410
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.367347
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.140090
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.060052
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.040438
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.049575
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.194496
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.218808
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.091971
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.266317
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.079110
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.193660
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.171428
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.242129
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.086190
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.056885


KeyboardInterrupt: 